# 0b Riluzole Relationships Between FDG and Plasma Biomarkers
adding boostrap to significant models

In [ ]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import numpy as np;
import pandas as pd;
from sklearn import preprocessing 
from sklearn.utils import resample
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import itertools
import pickle

import scipy; # for spearmann correlation



In [ ]:
sm.__version__

# read in and clean data

In [ ]:
data_folder_loc = '//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/'
code_folder_loc = '//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/Riluzole FDG/'

In [ ]:
# read in data
composite_rr, pons_rr, para_rr, new_para_rr, ras_rr = pd.read_pickle(code_folder_loc + '/output/01_newdata.pkl')

In [ ]:
all_data = new_para_rr #[~new_para_rr['ID_ADM'].isin(['APE-792_23', 'APE-792_35', 'APE-792_39', 'APE-792_65', 'APE-792_71', 'RIL_17', 'RIL_21', 'RIL_24', 'APE-792_9','APE-792_22', 'RIL_13'])].copy()#
data_name = 'new_para_rr'

In [ ]:
# read in results from normal analyses
results_new_para, temp, data4 = pd.read_pickle(code_folder_loc + '/output/04_data_new_para_rr.pkl')


In [ ]:
# orig has additional 
# 
# pons (1st) is missing 'L_Hip', 'R_Hip', 'Avg_Hip', 'Temp', 'FRONTAL-gm', 'Avg_MedOrbFrontal', 'Avg PCC'
# para (1st) is missing 'L_Hip', 'R_Hip', 'Avg_Hip', 'Temp', 'FRONTAL-gm', 'Avg_MedOrbFrontal'

# para (new) is missing 'L_Hip', 'R_Hip', 'Avg_Hip', 'Temp', 'Avg PCC', 'Avg_MedOrbFrontal'
# para (new) also has 'SensMot-gm', 'Paracentral-gm', 'RASref1-gm'

# ras (new) is missing 'L_Hip', 'R_Hip', 'Avg_Hip', 'Temp', 'Avg_MedOrbFrontal'
# ras (new) also has 'SensMot-gm', 'Paracentral-gm', 'RASref1-gm', 'CO', 'Graycer-gm', 'Vermis-gm'

In [ ]:
# define biomarker groups
FDG_columns = ['Avg_MedOrbFrontal', 'Graycer_gm', 'Avg_PCC', 'Avg_Hip', 'MTL_gm', 'CO', 'Temp', 'SensMot_gm', 'L_Hip',
 'Vermis_gm', 'FRONTAL_gm', 'AC_gm', 'Precun_gm', 'Par_gm', 'RASref1_gm', 'Temp_gm', 'Paracentral_gm', 'R_Hip', 'PostCing_gm']

plasma_columns = [ 'Ab40', 'Ab42', 'Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau181_Ab42']
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

cog_columns = ['MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']
vol_columns = ['VOL_Ventricles_Lz',
       'VOL_Ventricles_Rz', 'VOL_Putamen_Lz', 'VOL_Putamen_Rz',
       'VOL_ParaHip_Lz', 'VOL_ParaHip_Rz', 'VOL_Fusi_Lz', 'VOL_Fusi_Rz',
       'VOL_InfTemp_Lz', 'VOL_InfTemp_Rz', 'VOL_MidTemp_Lz', 'VOL_MidTemp_Rz',
       'VOL_SupTemp_Lz', 'VOL_SupTemp_Rz', 'VOL_Precun_Lz', 'VOL_Precun_Rz',
       'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_ParaPostCentr_Lz',
       'VOL_ParaPostCentr_Rz', 'VOL_SupraMarg_Lz', 'VOL_SupraMarg_Rz',
       'VOL_SupPar_Lz', 'VOL_SupPar_Rz', 'VOL_OrbitFront_Lz',
       'VOL_OrbitFront_Rz', 'VOL_Insula_Lz', 'VOL_Insula_Rz',
       'VOL_InfFront_Lz', 'VOL_InfFront_Rz', 'VOL_MidFront_Lz',
       'VOL_MidFront_Rz', 'VOL_SupFront_Lz', 'VOL_SupFront_Rz',
       'VOL_PrecFront_Lz', 'VOL_PrecFront_Rz', 'VOL_LatOcc_Lz',
       'VOL_LatOcc_Rz', 'VOL_Lingual_Lz', 'VOL_Lingual_Rz', 'VOL_Cuneus_Lz',
       'VOL_Cuneus_Rz', 'VOL_Pericalc_Lz', 'VOL_Pericalc_Rz',
       'VOL_AntCingulate_Lz', 'VOL_AntCingulate_Rz', 'VOL_PostCingulate_Lz',
       'VOL_PostCingulate_Rz', 'VOL_Entorhinal_Lz', 'VOL_Entorhinal_Rz',
       'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz', 'VOL_TotalGray_Lz',
       'VOL_TotalGray_Rz', 'VOL_LatTemp_Lz', 'VOL_LatTemp_Rz',
       'VOL_Parietal_Lz', 'VOL_Parietal_Rz', 'VOL_Frontal_Lz',
       'VOL_Frontal_Rz', 'VOL_InfMidTemp_Lz', 'VOL_InfMidTemp_Rz',
       'VOL_InfInsFrontal_Lz', 'VOL_InfInsFrontal_Rz', 'VOL_MidSupFrontal_Lz',
       'VOL_MidSupFrontal_Rz', 'VOL_Inf_Mid_Fus_Temp_Lz',
       'VOL_Inf_Mid_Fus_Temp_Rz', 'VOL_Precun_InfPar_Lz',
       'VOL_Precun_InfPar_Rz', 'VOL_Precun_InfPar_Supramarg_Lz',
       'VOL_Precun_InfPar_Supramarg_Rz', 'VOL_LatOccLingCun_Lz',
       'VOL_LatOccLingCun_Rz', 'VOL_InfParSupra_Lz', 'VOL_InfParSupra_Rz']

In [ ]:
# when results were save, the log10 was removed from plasma names (makes it easier to graph)
# for this though we want to use the actual log data since fitting models
results_new_para[['x_var', 'y_var']] = results_new_para[['x_var', 'y_var']].replace(dict(zip(plasma_columns, log10_plasma_columns)))

In [ ]:
print('any missing covariate data?')
all_data[all_data['timepoint'].isin(['base'])].dropna(subset = [ele for ele in FDG_columns if ele in all_data.columns], how = 'all')[['age', 'Education_years', 'sex', 'apoe4_carrier', 'race']].isna().value_counts().sort_index()

## make data subsets

In [ ]:
# make data subsets
# baseline and non-trial data
baseline_data = all_data[all_data['timepoint'].isin(['base', 'not part of trial'])]

# baseline only
baseline_trial_data = all_data[all_data['timepoint'].isin(['base'])]

# baseline only and only age 70-80
baseline_trial_data_70_80 = baseline_trial_data[(baseline_trial_data['age']<=80) & (baseline_trial_data['age']>=70)]

In [ ]:
def bootstrap_regression(significant_combos):
    biomarker_relationship_results_log10 = pd.DataFrame()
    biomarker_relationship_results_log10_975 = pd.DataFrame()
    biomarker_relationship_results_log10_025 = pd.DataFrame()

    covariate_p_threshold = 0.05

    for idx in range(0,len(significant_combos)):
        x_var = significant_combos.iloc[idx]['x_var']
        y_var = significant_combos.iloc[idx]['y_var']
        
        temp_df_baseline =  baseline_trial_data.copy().dropna(subset = [y_var, x_var])

        # z-score data
        temp_df_baseline[[y_var, x_var, 'Education_years', 'age']] = preprocessing.StandardScaler().fit_transform(temp_df_baseline[[y_var, x_var, 'Education_years', 'age']])

        # run 1 - check for significant covariates
        sig_pvals_start = ['age', 'Education_years', 'apoe4_carrier', 'sex', 'race', x_var]
        change = 1
        full_model = smf.ols(formula = y_var + ' ~ ' + ' + '.join(sig_pvals_start), data = temp_df_baseline).fit()
        temp_df_baseline['full_model_residuals'] = full_model.resid

        while change > 0:
            model_vars = ' + '.join(sig_pvals_start)        
            test_model = smf.ols(formula = y_var + ' ~ ' + model_vars, data = temp_df_baseline).fit()
            sig_pvals = test_model.pvalues[test_model.pvalues < covariate_p_threshold].index.to_list()
            sig_pvals = [ele.split('[')[0] for ele in sig_pvals] # drop the [] indicators
            sig_pvals_unique = []
            for item in sig_pvals:
                if (item not in sig_pvals_unique) & (item not in ['Intercept']): sig_pvals_unique.append(item)
            if (x_var not in sig_pvals_unique):
                sig_pvals_unique = sig_pvals_unique + [x_var]
            change = len(sig_pvals_start) - len(sig_pvals_unique)
            sig_pvals_start = sig_pvals_unique
        
        # run 2 -with just the significant covariates (but forcing the x_var to be included)
        model_vars = ' + '.join(sig_pvals_unique)  

        resampled_df = pd.DataFrame()
        for resample_idx in range(0,1000):
            # multivariate linear model with age and education years as covariates
            resampled_data = resample(temp_df_baseline, replace=True, n_samples=None, random_state=resample_idx, stratify=None)
            model = smf.ols(formula = y_var + ' ~ ' + model_vars , data = resampled_data).fit()
            resampled_result = pd.DataFrame([model.pvalues, model.params])
            resampled_result['index'] = ['p', 'B']
            resampled_result.index = [0,0]
            resampled_result = resampled_result.pivot(columns = ['index'])
            resampled_result.columns = [f'{y}_{x}' for x,y in resampled_result.columns]
            resampled_result
            resampled_df = pd.concat([resampled_df, resampled_result])

        model_n_baseline = len(temp_df_baseline.dropna(subset = sig_pvals_unique + [y_var]))



        # save average results
        temp_results  = pd.DataFrame({'x_var':[y_var], 'y_var':[x_var], 'model_n_baseline':[model_n_baseline]})
        temp_results  = pd.DataFrame(resampled_df.quantile(q=0.5)).T
        temp_results['x_var'], temp_results['y_var'], temp_results['model_n_baseline'] = y_var, x_var, model_n_baseline        
        temp_results = temp_results.rename(columns = dict(zip(temp_results.columns, [ele.replace(x_var, 'y_var') \
            for ele in temp_results.columns ])))         # rename so var2 is in the same column
        biomarker_relationship_results_log10 = pd.concat([biomarker_relationship_results_log10, 
        temp_results]) # add to full list of results

        # save 95% results
        temp_results  = pd.DataFrame({'x_var':[y_var], 'y_var':[x_var], 'model_n_baseline':[model_n_baseline]})
        temp_results  = pd.DataFrame(resampled_df.quantile(q=0.975)).T
        temp_results['x_var'], temp_results['y_var'], temp_results['model_n_baseline'] = y_var, x_var, model_n_baseline        
        temp_results = temp_results.rename(columns = dict(zip(temp_results.columns, [ele.replace(x_var, 'y_var') \
            for ele in temp_results.columns ])))         # rename so var2 is in the same column
        biomarker_relationship_results_log10_975 = pd.concat([biomarker_relationship_results_log10_975, 
        temp_results]) # add to full list of results


        # save 5% results
        temp_results  = pd.DataFrame({'x_var':[y_var], 'y_var':[x_var], 'model_n_baseline':[model_n_baseline]})
        temp_results  = pd.DataFrame(resampled_df.quantile(q=0.025)).T
        temp_results['x_var'], temp_results['y_var'], temp_results['model_n_baseline'] = y_var, x_var, model_n_baseline        
        temp_results = temp_results.rename(columns = dict(zip(temp_results.columns, [ele.replace(x_var, 'y_var') \
            for ele in temp_results.columns ])))         # rename so var2 is in the same column
        biomarker_relationship_results_log10_025 = pd.concat([biomarker_relationship_results_log10_025, 
        temp_results]) # add to full list of results

        p9.options.figure_size = (4,3)    
        if len(resampled_df['p_' + x_var].dropna()) > 0:
            title = y_var + ': original B=' + str(round(significant_combos.iloc[idx]['model_B_val_x_var'], 2))
            subtitle =  'bootstrap B=' + str(round(resampled_df['B_' + x_var].quantile(q=0.5), 2)) + ' (' + resampled_df['B_' + x_var].quantile(q=0.025).round(2).astype('str') + ' to ' + resampled_df['B_' + x_var].quantile(q=0.975).round(2).astype('str') + ')'
            plot = (
                    p9.ggplot(resampled_df, p9.aes(x = 'B_' + x_var))
                    + p9.theme_bw(base_size = 11)            
                    + p9.geom_histogram(size = 0.5, bins = 20)
                    + p9.geom_vline(xintercept = resampled_df['B_' + x_var].quantile(q=0.025), color = 'red')
                    + p9.geom_vline(xintercept = resampled_df['B_' + x_var].quantile(q=0.975), color = 'red')
                    + p9.geom_vline(xintercept = resampled_df['B_' + x_var].quantile(q=0.5), color = 'blue')   
                    + p9.labs(title = title, subtitle = subtitle)   
                    + p9.scale_x_continuous(breaks = np.arange(round(resampled_df['B_' + x_var].min(),1), round(resampled_df['B_' + x_var].max(), 1), 0.2).round(1))
                )
            print(plot)    
            title = y_var + ': original p=' + str(round(significant_combos.iloc[idx]['model_p_val_x_var'], 3))
            subtitle =  'bootstrap p=' + str(round(resampled_df['p_' + x_var].quantile(q=0.5), 3)) + ' (' + resampled_df['p_' + x_var].quantile(q=0.025).round(3).astype('str') + ' to ' + resampled_df['p_' + x_var].quantile(q=0.975).round(3).astype('str') + ')'
            plot = (
                    p9.ggplot(resampled_df, p9.aes(x = 'p_' + x_var))
                    + p9.theme_bw(base_size = 11)            
                    + p9.geom_histogram(size = 0.5, bins = 20)
                    + p9.geom_vline(xintercept = resampled_df['p_' + x_var].quantile(q=0.025), color = 'red')
                    + p9.geom_vline(xintercept = resampled_df['p_' + x_var].quantile(q=0.975), color = 'red')
                    + p9.geom_vline(xintercept = resampled_df['p_' + x_var].quantile(q=0.5), color = 'blue')   
                    + p9.labs(title = title, subtitle = subtitle)   
                    + p9.scale_x_continuous(breaks = np.arange(round(resampled_df['p_' + x_var].min(),2), round(resampled_df['p_' + x_var].max(), 2), 0.1).round(2))
                    )
            print(plot)    

            
    # remove self-correlations
    biomarker_relationship_results_log10 = biomarker_relationship_results_log10[~(biomarker_relationship_results_log10['y_var'] == biomarker_relationship_results_log10['x_var'])]
    biomarker_relationship_results_log10_975 = biomarker_relationship_results_log10_975[~(biomarker_relationship_results_log10_975['y_var'] == biomarker_relationship_results_log10_975['x_var'])]
    biomarker_relationship_results_log10_025 = biomarker_relationship_results_log10_025[~(biomarker_relationship_results_log10_025['y_var'] == biomarker_relationship_results_log10_025['x_var'])]

    return biomarker_relationship_results_log10,  biomarker_relationship_results_log10_975, biomarker_relationship_results_log10_025

# Analyses

In [ ]:
biomarker_relationship_results_log10 = pd.DataFrame()
biomarker_relationship_results_log10_975 = pd.DataFrame()
biomarker_relationship_results_log10_025 = pd.DataFrame()

significant_combos = results_new_para.copy()
significant_combos = significant_combos[significant_combos['x_var'].isin(['Ab42_40_log10', 'GFAP_log10', 'NFL_log10', 'pTau181_log10', 'pTau217_log10', 'pTau231_log10'])]
significant_combos = significant_combos[~significant_combos['y_var'].isin(['RASref1_gm', 'Paracentral_gm', 'Ab42_log10', 'pTau181_Ab42_log10', 'Ab40_log10'])]

significant_combos = significant_combos[significant_combos['model_p_val_x_var']<0.05].reset_index(drop = True)
significant_combos

# Ab42/40 bootstrapping

In [ ]:
a,  b, c = bootstrap_regression(significant_combos[significant_combos['x_var'].isin(['Ab42_40_log10'])])
biomarker_relationship_results_log10 = pd.concat([biomarker_relationship_results_log10, a])
biomarker_relationship_results_log10_975 = pd.concat([biomarker_relationship_results_log10_975, b])
biomarker_relationship_results_log10_025 = pd.concat([biomarker_relationship_results_log10_025, c])

# GFAP bootstrapping

In [ ]:
a,  b, c = bootstrap_regression(significant_combos[significant_combos['x_var'].isin(['GFAP_log10'])])
biomarker_relationship_results_log10 = pd.concat([biomarker_relationship_results_log10, a])
biomarker_relationship_results_log10_975 = pd.concat([biomarker_relationship_results_log10_975, b])
biomarker_relationship_results_log10_025 = pd.concat([biomarker_relationship_results_log10_025, c])

# NFL bootstrapping

In [ ]:
a,  b, c = bootstrap_regression(significant_combos[significant_combos['x_var'].isin(['NFL_log10'])])
biomarker_relationship_results_log10 = pd.concat([biomarker_relationship_results_log10, a])
biomarker_relationship_results_log10_975 = pd.concat([biomarker_relationship_results_log10_975, b])
biomarker_relationship_results_log10_025 = pd.concat([biomarker_relationship_results_log10_025, c])

# pTau217 bootstrapping

In [ ]:
a,  b, c = bootstrap_regression(significant_combos[significant_combos['x_var'].isin(['pTau217_log10'])])
biomarker_relationship_results_log10 = pd.concat([biomarker_relationship_results_log10, a])
biomarker_relationship_results_log10_975 = pd.concat([biomarker_relationship_results_log10_975, b])
biomarker_relationship_results_log10_025 = pd.concat([biomarker_relationship_results_log10_025, c])

# pTau181 bootstrapping

In [ ]:
a,  b, c = bootstrap_regression(significant_combos[significant_combos['x_var'].isin(['pTau181_log10'])])
biomarker_relationship_results_log10 = pd.concat([biomarker_relationship_results_log10, a])
biomarker_relationship_results_log10_975 = pd.concat([biomarker_relationship_results_log10_975, b])
biomarker_relationship_results_log10_025 = pd.concat([biomarker_relationship_results_log10_025, c])

# pTau231 bootstrapping

In [ ]:
a,  b, c = bootstrap_regression(significant_combos[significant_combos['x_var'].isin(['pTau231_log10'])])
biomarker_relationship_results_log10 = pd.concat([biomarker_relationship_results_log10, a])
biomarker_relationship_results_log10_975 = pd.concat([biomarker_relationship_results_log10_975, b])
biomarker_relationship_results_log10_025 = pd.concat([biomarker_relationship_results_log10_025, c])

# Other select models

In [ ]:
select_combos = results_new_para.copy()
select_combos = select_combos[\
    (select_combos['x_var'].isin(['pTau217_log10']) & select_combos['y_var'].isin(['MTL_gm']) ) | \
    (select_combos['x_var'].isin(['GFAP_log10']) & select_combos['y_var'].isin(['Precun_gm', 'Par_gm']) ) \
]
    #| \
    #(select_combos['x_var'].isin(['pTau217_log10']) & select_combos['y_var'].isin(['pTau181_log10', 'pTau231_log10']) ) | \
    #(select_combos['x_var'].isin(['pTau231_log10']) & select_combos['y_var'].isin(['pTau181_log10']) ) | \
    #]
a,  b, c = bootstrap_regression(select_combos)


# Clean up full list of results

In [ ]:
biomarker_relationship_results_log10['type'] = 'other'
biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['y_var'].isin( [ele for ele in FDG_columns if ele in all_data.columns]), 'type'] = 'FDG PET'
biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['y_var'].isin(log10_plasma_columns + log10_plasma_columns), 'type'] = 'Plasma'
biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['y_var'].isin(cog_columns), 'type'] = 'Cognitive'
biomarker_relationship_results_log10.loc[biomarker_relationship_results_log10['y_var'].isin(vol_columns), 'type'] = 'Volumetric'